In [ ]:
pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 3.3 MB/s eta 0:00:00


In [ ]:
from groq import Groq

client = Groq(api_key="")

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the concept of AI in a sentence.",
        }
    ],
    model="llama-3.3-70b-versatile",
    temperature=0,
    seed=123,
    max_completion_tokens=2048,
)

def chamar_llm(prompt):
    response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=2048
    )
    return response.choices[0].message.content


In [ ]:
from google.colab import files
import json


uploaded = files.upload()

filename = list(uploaded.keys())[0]

with open(filename, 'r') as f:
    data = json.load(f)

print(f"JSON carregado! Total de itens: {len(data)}")

Saving resultados_data_small.json to resultados_data_small.json
JSON carregado! Total de itens: 30


In [ ]:
import json
from tqdm import tqdm

# file_path = "instruction-data-with-response.json"

# with open(file_path, "r") as file:
#     test_data = json.load(file)


def format_input(entry):
    instruction_text = (
        f"Below is an instruction that describes a task. "
        f"Write a response that appropriately completes the request."
        f"\n\n### Instruction:\n{entry['instruction']}"
    )

    input_text = f"\n\n### Input:\n{entry['input']}" if entry["input"] else ""

    return instruction_text + input_text

In [ ]:
import json

responses = []
for entry in data:
    prompt = (
        f"Given the input `{format_input(entry)}` "
        f"and correct output `{entry['output']}`, "
        f"score the base model (A) response `{entry['base_model_response']}`,"
        f"and the fine-tuned model (B) response `{entry['model_response']}`"
        f"on a scale from 0 to 5, where 5 has the best factual correction;"
        f"on another scale from 0 to 5, where 5 has the best adherence to instruction;"
        f"and finally on a scale from 0 to 5, where 5 has the best clarity/usefulness."
        "At the end, choose a winner (A or B, or draw) and justify why."
        "Return ONLY a valid JSON object with this structure:\n"
        "{"
        '  "A": {"factual_correction": score, "adherence": score, "clarity": score},'
        '  "B": {"factual_correction": score, "adherence": score, "clarity": score},'
        '  "Winner": {"winner": winner, "why": "explanation"}'
        "}\n"
        "Return ONLY a Python string, no explanation, no extra text"
    )
    response = chamar_llm(prompt)
    responses.append(json.loads(response))


In [ ]:
len(responses)

30

In [ ]:
with open('judge-small-dataset.json', 'w', encoding='utf-8') as f:
     json.dump(responses, f, indent=4, ensure_ascii=False)
files.download("judge-small-dataset.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
print(responses[0])

{'A': {'factual_correction': 0, 'adherence': 0, 'clarity': 0}, 'B': {'factual_correction': 5, 'adherence': 5, 'clarity': 5}, 'Winner': {'winner': 'B', 'why': 'Model B accurately identifies the artist responsible for the mural, adheres to the instruction, and provides a clear and useful response.'}}
